In [1]:
import logging

from rlduels.src.primitives.trajectory_pair import Transition, Trajectory, TrajectoryPair, NDArray

from rlduels.src.database.database_manager import MongoDBManager

from rlduels.src.create_video import create_videos_from_pair

from rlduels.src.env_wrapper import EnvWrapper, GymWrapper

logging.basicConfig(level=logging.DEBUG,
                    format='%(asctime)s - %(levelname)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S')

In [2]:
from rlduels.src.utils.simulate import simulate_trajectory_pairs

tps = simulate_trajectory_pairs(env="CartPole-v1", n=1)

Episode 1: Total reward = 71.0
Episode 2: Total reward = 11.0


In [3]:
db = MongoDBManager(test=False)

...


In [4]:
x = tps[0]
print(db.add_entry(x))
print(db.find_entry(x))
print(db.delete_entry(x))
print(db.find_entry(x))

2024-05-13 01:34:20 - DEBUG - {"message": "Server selection started", "selector": "<function writable_server_selector at 0x791c5cefce50>", "operation": "insert", "topologyDescription": "<TopologyDescription id: 664151eb729808db8eb38548, topology_type: Single, servers: [<ServerDescription ('localhost', 27017) server_type: Standalone, rtt: 0.0004241530004946981>]>", "clientId": {"$oid": "664151eb729808db8eb38548"}}
2024-05-13 01:34:20 - DEBUG - {"message": "Server selection succeeded", "selector": "<function writable_server_selector at 0x791c5cefce50>", "operation": "insert", "topologyDescription": "<TopologyDescription id: 664151eb729808db8eb38548, topology_type: Single, servers: [<ServerDescription ('localhost', 27017) server_type: Standalone, rtt: 0.0004241530004946981>]>", "clientId": {"$oid": "664151eb729808db8eb38548"}, "serverHost": "localhost", "serverPort": 27017}
2024-05-13 01:34:20 - DEBUG - {"clientId": {"$oid": "664151eb729808db8eb38548"}, "message": "Command started", "comm

('Added 79258707-52fc-4824-95f5-dcb1f44c5fa2 to the database', None)
None
('Entry with ID 79258707-52fc-4824-95f5-dcb1f44c5fa2 successfully deleted.', None)
None


In [8]:
db.close_db()

2024-05-06 21:36:03 - DEBUG - Deleting every entry from the database.
2024-05-06 21:36:03 - INFO - Database closed successfully.


In [9]:
import json

serialized_data = x.json()

data_to_store = json.loads(serialized_data)

data_to_store['_id'] = data_to_store.pop('id')

print(data_to_store)

{'trajectory1': {'env_name': 'CartPole-v1', 'information': {'seed': 42}, 'transitions': [{'state': {'array': [0.02739560417830944, -0.006112155970185995, 0.03585979342460632, 0.019736802205443382]}, 'action': {'array': 0}, 'next_state': {'array': [0.02727336250245571, -0.20172953605651855, 0.036254528909921646, 0.32351475954055786]}, 'reward': 1.0, 'terminated': False, 'truncated': False}, {'state': {'array': [0.02727336250245571, -0.20172953605651855, 0.036254528909921646, 0.32351475954055786]}, 'action': {'array': 0}, 'next_state': {'array': [0.02323877066373825, -0.39734846353530884, 0.04272482171654701, 0.6274068355560303]}, 'reward': 1.0, 'terminated': False, 'truncated': False}, {'state': {'array': [0.02323877066373825, -0.39734846353530884, 0.04272482171654701, 0.6274068355560303]}, 'action': {'array': 0}, 'next_state': {'array': [0.015291801653802395, -0.593039870262146, 0.05527295917272568, 0.933233380317688]}, 'reward': 1.0, 'terminated': False, 'truncated': False}, {'state':

# Test BufferedQueueManager

In [1]:
import logging

from rlduels.src.primitives.trajectory_pair import Transition, Trajectory, TrajectoryPair, NDArray

from rlduels.src.database.database_manager import MongoDBManager

from rlduels.src.create_video import create_videos_from_pair

from rlduels.src.env_wrapper import EnvWrapper, GymWrapper

from rlduels.src.buffered_queue_manager import BufferedQueueManager


logging.basicConfig(level=logging.DEBUG,
                    format='%(asctime)s - %(levelname)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S')

In [2]:
import gymnasium as gym

def run_cartpole():
    # Create the CartPole environment
    env = GymWrapper.create_env(name='CartPole-v1', render_mode="rgb_array")

    num_episodes = 10

    trajectories = []

    for episode in range(num_episodes):
        # Reset the environment for a new episode
        seed = 42
        observation, info = env.reset(seed=42)
        total_reward = 0
        done = False

        transitions = []

        while not done:
            env.render()

            action = env.sample_action()

            next_observation, reward, terminated, truncated, info = env.step(action)

            transitions.append(Transition.create(
                state=observation,
                action=action,
                reward=reward,
                terminated=terminated,
                truncated=truncated,
                next_state=next_observation
            ))

            observation = next_observation
            total_reward += reward
            done = terminated or truncated
        
        trajectories.append(Trajectory(
            env_name = env.name,
            information={'seed': 42},
            transitions=transitions
        ))

        if done:
            print(f"Episode {episode + 1}: Total reward = {total_reward}")

    # Close the environment
    env.close()

    # Return trajectories for further analysis if necessary
    return trajectories

In [3]:
db = MongoDBManager()

In [4]:
trajs = run_cartpole()

for i in range(len(trajs)):
    if i != (len(trajs)-1):
        print(db.add_entry(TrajectoryPair(trajectory1=trajs[i], trajectory2=trajs[i+1])))

Episode 1: Total reward = 22.0
Episode 2: Total reward = 23.0
Episode 3: Total reward = 10.0
Episode 4: Total reward = 19.0
Episode 5: Total reward = 15.0
Episode 6: Total reward = 12.0
Episode 7: Total reward = 33.0


2024-05-07 00:15:17 - DEBUG - {"message": "Server selection started", "selector": "<function writable_server_selector at 0x7809affd3670>", "operation": "insert", "topologyDescription": "<TopologyDescription id: 6639567541eddcc0b49132b7, topology_type: Single, servers: [<ServerDescription ('localhost', 27017) server_type: Standalone, rtt: 0.0005299440017552115>]>", "clientId": {"$oid": "6639567541eddcc0b49132b7"}}
2024-05-07 00:15:17 - DEBUG - {"message": "Server selection succeeded", "selector": "<function writable_server_selector at 0x7809affd3670>", "operation": "insert", "topologyDescription": "<TopologyDescription id: 6639567541eddcc0b49132b7, topology_type: Single, servers: [<ServerDescription ('localhost', 27017) server_type: Standalone, rtt: 0.0005299440017552115>]>", "clientId": {"$oid": "6639567541eddcc0b49132b7"}, "serverHost": "localhost", "serverPort": 27017}
2024-05-07 00:15:17 - DEBUG - {"clientId": {"$oid": "6639567541eddcc0b49132b7"}, "message": "Command started", "comm

Episode 8: Total reward = 27.0
Episode 9: Total reward = 15.0
Episode 10: Total reward = 18.0
('Added 819831f9-4558-45c2-bdda-ef59ebc2b98b to the database', None)
('Added d2b8bbfb-d290-45d7-97d9-f160c7b79176 to the database', None)
('Added b6d04e59-c50d-461f-ae7e-55a1ba206a5d to the database', None)
('Added d49a667d-b3f1-4fb9-8ba1-046f167e68c4 to the database', None)
('Added 10766def-5725-436b-b026-d54ac6ecce1d to the database', None)
('Added 30fa5f58-ceb8-443b-8fad-6056a2a9c1d9 to the database', None)
('Added 8c747886-c8af-4ab3-8c8c-16b3f4df4974 to the database', None)
('Added 34287f55-c63f-4888-a9e9-265c643e279d to the database', None)
('Added 5760eab6-6834-4aca-a2cf-9b3bb6c146bb to the database', None)


In [5]:
entry, error = db.get_next_entry()
print(error)
print(type(entry['trajectory1']['transitions'][0]['state']['array']))

2024-05-07 00:15:17 - DEBUG - Getting next entry.
2024-05-07 00:15:17 - DEBUG - {"message": "Server selection started", "selector": "Primary()", "operation": "find", "topologyDescription": "<TopologyDescription id: 6639567541eddcc0b49132b7, topology_type: Single, servers: [<ServerDescription ('localhost', 27017) server_type: Standalone, rtt: 0.0005299440017552115>]>", "clientId": {"$oid": "6639567541eddcc0b49132b7"}}
2024-05-07 00:15:17 - DEBUG - {"message": "Server selection succeeded", "selector": "Primary()", "operation": "find", "topologyDescription": "<TopologyDescription id: 6639567541eddcc0b49132b7, topology_type: Single, servers: [<ServerDescription ('localhost', 27017) server_type: Standalone, rtt: 0.0005299440017552115>]>", "clientId": {"$oid": "6639567541eddcc0b49132b7"}, "serverHost": "localhost", "serverPort": 27017}
2024-05-07 00:15:17 - DEBUG - {"clientId": {"$oid": "6639567541eddcc0b49132b7"}, "message": "Command started", "command": "{\"find\": \"videos\", \"sort\": {\

<class 'list'>


TypeError: from_dict() got an unexpected keyword argument 'state'

In [5]:
buffered_queue = BufferedQueueManager(db_manager=db, n=2, daemon=False)

2024-05-07 00:05:09 - DEBUG - Getting next entry.
2024-05-07 00:05:09 - DEBUG - {"message": "Server selection started", "selector": "Primary()", "operation": "find", "topologyDescription": "<TopologyDescription id: 6639541316586b09482f2f70, topology_type: Single, servers: [<ServerDescription ('localhost', 27017) server_type: Standalone, rtt: 0.0005682510018232279>]>", "clientId": {"$oid": "6639541316586b09482f2f70"}}
2024-05-07 00:05:09 - DEBUG - {"message": "Server selection succeeded", "selector": "Primary()", "operation": "find", "topologyDescription": "<TopologyDescription id: 6639541316586b09482f2f70, topology_type: Single, servers: [<ServerDescription ('localhost', 27017) server_type: Standalone, rtt: 0.0005682510018232279>]>", "clientId": {"$oid": "6639541316586b09482f2f70"}, "serverHost": "localhost", "serverPort": 27017}
2024-05-07 00:05:09 - DEBUG - {"clientId": {"$oid": "6639541316586b09482f2f70"}, "message": "Command started", "command": "{\"find\": \"videos\", \"sort\": {\

KeyboardInterrupt: 